<a href="https://colab.research.google.com/github/zychen1017/Building-Blocks/blob/main/optuna_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.4 MB/s eta 0:00:00


In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import optuna
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [ ]:
healthexp = sns.load_dataset('healthexp')
healthexp.head(10)

,Year,Country,Spending_USD,Life_Expectancy
0,1970,Germany,252.311,70.6
1,1970,France,192.143,72.2
2,1970,Great Britain,123.993,71.9
3,1970,Japan,150.437,72.0
4,1970,USA,326.961,70.9
5,1971,Canada,313.391,72.8
6,1971,Germany,298.251,70.8
7,1971,Great Britain,134.172,71.9
8,1971,Japan,163.854,72.9
9,1971,USA,357.988,71.2


In [ ]:
healthexp = pd.get_dummies(healthexp)

In [ ]:
X = healthexp.drop(['Life_Expectancy'], axis=1)
y = healthexp['Life_Expectancy']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

In [ ]:
rfr = RandomForestRegressor(random_state=13)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)

In [ ]:
mean_absolute_error(y_test, y_pred)

0.25916363636361917

In [ ]:
mean_squared_error(y_test, y_pred)

0.10221141818181628

In [ ]:
r2_score(y_test, y_pred)

0.9910457602615238

# create function for turning


In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 10, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 32)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32)

    model = RandomForestRegressor(n_estimators=n_estimators,
                    max_depth=max_depth,
                    min_samples_split=min_samples_split,
                    min_samples_leaf=min_samples_leaf)

    score = cross_val_score(model, X, y, n_jobs=-1, cv=5, scoring='neg_mean_squared_error').mean()
    return score

In [ ]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=42))

[I 2024-07-07 17:43:53,591] A new study created in memory with name: no-name-bcc876c1-cfb6-4eb5-ad5d-3f1b433a851b


In [ ]:
study.optimize(objective, n_trials=50)

[I 2024-07-07 17:45:14,157] Trial 10 finished with value: -5.3619990842620755 and parameters: {'n_estimators': 209, 'max_depth': 30, 'min_samples_split': 3, 'min_samples_leaf': 30}. Best is trial 3 with value: -2.986136132615743.
[I 2024-07-07 17:45:16,740] Trial 11 finished with value: -4.040065718675727 and parameters: {'n_estimators': 333, 'max_depth': 37, 'min_samples_split': 11, 'min_samples_leaf': 17}. Best is trial 3 with value: -2.986136132615743.
[I 2024-07-07 17:45:22,511] Trial 12 finished with value: -4.972418371060651 and parameters: {'n_estimators': 592, 'max_depth': 17, 'min_samples_split': 32, 'min_samples_leaf': 25}. Best is trial 3 with value: -2.986136132615743.
[I 2024-07-07 17:45:27,858] Trial 13 finished with value: -5.324191340217823 and parameters: {'n_estimators': 946, 'max_depth': 46, 'min_samples_split': 20, 'min_samples_leaf': 30}. Best is trial 3 with value: -2.986136132615743.
[I 2024-07-07 17:45:28,947] Trial 14 finished with value: -3.8140387128951594 an

# find the best params and score

In [ ]:
best_params = study.best_params
best_score = study.best_value
print(f"Best Hyperparameters: {best_params}")
print(f"Best Accuracy: {best_score:.3f}")

Best Hyperparameters: {'n_estimators': 570, 'max_depth': 27, 'min_samples_split': 2, 'min_samples_leaf': 4}
Best Accuracy: -2.419


# trial visualization

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_slice(study, params=['n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split'])

In [ ]:
optuna.visualization.plot_param_importances(study)

# find each best parameters

In [ ]:
best_n_estimators = best_params['n_estimators']
best_max_depth = best_params['max_depth']
best_min_samples_split = best_params['min_samples_split']
best_min_samples_leaf = best_params['min_samples_leaf']

# use best parameters train model

In [ ]:
best_model = RandomForestRegressor(n_estimators=best_n_estimators, max_depth=best_max_depth, min_samples_split=best_min_samples_split, min_samples_leaf=best_min_samples_leaf)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

In [ ]:
mean_absolute_error(y_test, y_pred)

0.47995931922376345

In [ ]:
mean_squared_error(y_test, y_pred)

0.3138979244006099

In [ ]:
r2_score(y_test, y_pred)

0.9725009463865047

# use dictionary, it is more convenient.

In [ ]:
def objective(trial):

  #define hyperparameters to optimize for
  params = {
      'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
      'max_depth': trial.suggest_int('max_depth', 3, 10),
      'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 1),
      'subsample': trial.suggest_uniform('subsample', 0.1, 1),
      'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1),
      #'gamma': trial.suggest_uniform('gamma', 0, 1),
      'alpha': trial.suggest_loguniform('alpha', 2, 5),
      'lambda': trial.suggest_loguniform('lambda', 2, 5),
      'min_child_weight': trial.suggest_int('min_child_weight', 1, 300)
  }

  model = XGBClassifier(**params, random_state=0)


  score = cross_val_score(model, X, Y, cv=5).mean()

  return score

In [ ]:
#run hyperparameter optimization with optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2)#n_trials=2 is just for demonstration purpose
best_params = study.best_params
print(f'Best hyperparameters: {best_params}')

In [ ]:
best_params= {'n_estimators': 465,
              'max_depth': 4,
              'learning_rate': 0.13566308331651933,
              'subsample': 0.6217729332313746,
              'colsample_bytree': 0.9406231273240503,
              'alpha': 3.8470299829756747,
              'lambda': 2.6061951769367186,
              'min_child_weight': 6}
xgb_tuned= XGBClassifier(**best_params, random_state=0)